### Relax Inc. mock takehome challenge
Task: Defining  an  "adopted  user"   as  a  user  who   has  logged  into  the  product  on  three  separate
days  in  at  least  one  seven­day  period ,  identify  which  factors  predict  future  user
adoption .

In [28]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from datetime import datetime

#### Read files

In [2]:
engagement_file = "takehome_user_engagement.csv"
users_file = "takehome_users2.csv"

In [50]:
engagement = pd.DataFrame(pd.read_csv(engagement_file, parse_dates=True))
users = pd.DataFrame(pd.read_csv(users_file, parse_dates=True, encoding = 'latin1'))

In [30]:
users[:3]

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,4/22/2014 3:53,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,11/15/2013 3:45,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,3/19/2013 23:14,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0


In [32]:
engagement[:3]

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1


In [51]:
engagement['time_stamp'] = engagement.time_stamp.apply(lambda d: datetime.strptime(d, '%Y-%m-%d %H:%M:%S'))

In [52]:
engagement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
time_stamp    207917 non-null datetime64[ns]
user_id       207917 non-null int64
visited       207917 non-null int64
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.8 MB


In [146]:
engagement['date']= engagement['time_stamp'].apply(lambda s: s.date())
engagement.drop(['visited','time_stamp'], axis=1, inplace=True)

In [147]:
engagement.head()

,user_id,date
0,1,2014-04-22
1,2,2013-11-15
2,2,2013-11-29
3,2,2013-12-09
4,2,2013-12-25


#### Adopted users
Identify which users have logged into the product on 3 days in one 7-day period.
<br>
use engagement table -- ```user_id``` == ```object_id```

Strategy: 
- count visits per day by user
- check there aren't multiple visits/day
- use timedelta to calculate difference between visits, and check if <= 7

In [105]:
grouped = engagement.groupby(['date','user_id'])[['visited']].sum()
grouped.head()

visited
date       user_id         
2012-05-31 1693           1
           3428           1
           9899           1
           10012          1
2012-06-01 1995           1

In [97]:
grouped['visited'][grouped['visited']>1]

Series([], Name: visited, dtype: int64)

There are no instances of more than one visited per day (empty series)

In [116]:
grouped.reset_index(inplace=True)
grouped.head()

,level_0,index,date,user_id,visited
0,0,0,2012-05-31,1693,1
1,1,1,2012-05-31,3428,1
2,2,2,2012-05-31,9899,1
3,3,3,2012-05-31,10012,1
4,4,4,2012-06-01,1995,1


In [123]:
visits = grouped[['user_id','date','visited']].sort_values('user_id')
visits.head()

,user_id,date,visited
180225,1,2014-04-22,1
121083,2,2014-01-08,1
139947,2,2014-02-13,1
117089,2,2013-12-31,1
102020,2,2013-11-29,1


In [144]:
total_visits = visits.groupby([,'date','user_id'])[['visited']].sum()
print('time agnositc, the number of people who visited more than 3 times is', 
      len(total_visits[total_visits.visited > 3]))

time agnositc, the number of people who visited more than 3 times is 0


In [145]:
total_visits.head()

visited
user_id date               
1       2014-04-22        1
2       2013-11-15        1
        2013-11-29        1
        2013-12-09        1
        2013-12-25        1

In [140]:
total_visits[total_visits.visited > 3][:10]

,visited
user_id,
2,14
10,284
20,7
33,18
42,342
43,8
50,17
53,9
60,8
